In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import pickle

In [3]:
# Load preprocessed data
df = pd.read_csv('../data/processed/train_data.csv')

In [4]:
# Data cleaning
df['review'] = df['review'].str.lower().str.replace('[^\w\s]', '')

# Convert sentiment labels to integers
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

C:\Users\asus\AppData\Local\Temp\ipykernel_11116\3128365610.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review'] = df['review'].str.lower().str.replace('[^\w\s]', '')


In [5]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
padded_sequences = pad_sequences(sequences, maxlen=200)

In [7]:
# Save tokenizer
with open('../models/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Split data
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, df['sentiment'], test_size=0.2)

# Define model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=200),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    LSTM(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=32)

# Save model
model.save('models/sentiment_model.h5')

1000/1000 [==============================] - 220s 214ms/step - loss: 0.3617 - accuracy: 0.8460 - val_loss: 0.3341 - val_accuracy: 0.8660
